# Using Hugging Face Inference with KDB.AI to Create a AI Tool Search Engine

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

How to get started with using the Huggingface Inference API with KDB.AI.

You will learn how to:

1. Connect to KDB.AI
1. Create a KDB.AI Table
1. Add Data to the KDB.AI Table
1. Perform Similarity Search using the Huggingface Inference API

# Why Use Hugging Face for Embeddings?

When building production applications that utilize embeddings, it's often advantageous to use open-source embedding models for several reasons:

1. **Control**: Open-source models give developers more control over the embeddings process, reducing dependence on third-party embedding providers.

2. **Local Embedding**: With open-source models, you can create embeddings locally, which is particularly useful for embedding your dataset.

A common approach is to use a Python framework like sentence-transformers, developed by Hugging Face, which offers state-of-the-art sentence, text, and image embeddings. Here's a typical workflow:

1. **Embed your dataset locally**: Use a library like FastEmbed (built on top of Hugging Face's transformers library, optimized for speed) to embed your dataset, which might consist of AI tools and associated metadata.

2. **Embed queries at inference time**: When a user submits a query, use an external service like Hugging Face's Inference API to embed the query. This eliminates the need to deploy your own model, allowing you to leverage a fully optimized external service.

By following this approach, you can build a system that searches through hundreds of AI tools without the need to deploy any infrastructure (and scale to millions!). Additionally, since you embed the dataset locally, you can use Hugging Face's free plan without requiring a credit card or worrying about hitting rate limits, at least until you are ready for production.

In this tutorial, we will walk through the process of embedding a dataset of AI tools using FastEmbed, and then using Hugging Face's Inference API to embed queries at inference time, enabling efficient and scalable search capabilities.

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

In [1]:
!pip install kdbai_client fastembed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.40.0 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is inco

### Import Packages

In [2]:
# vector DB
import os
from getpass import getpass
import kdbai_client as kdbai
import time

In [3]:
import numpy as np
import pandas as pd

## 1. Connect to KDB.AI

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [11]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

KDB.AI endpoint: https://cloud.kdb.ai/instance/wrve8kwshj
KDB.AI API key: ··········


In [5]:
HF_TOKEN = (
    os.environ["HF_TOKEN"]
    if "HF_TOKEN" in os.environ
    else getpass("Hugging Face token: ")
)

Hugging Face token: ··········


In [12]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Verify Defined Tables

We can check our connection using the `session.list()` function.
This will return a list of all the tables we have defined in our vector database thus far.
This should return an empty list.

In [13]:
# ensure no table called "ai_tools" exists
try:
    session.table("ai_tools").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [15]:
session.list()

['transcripts',
 'transcripts_LHX',
 'market_data_15',
 'eiffel_tower',
 'documents',
 'paul_graham',
 'documents1',
 'mytable']

## 2. Create a KDB.AI Table

To create a table we can use `create_table`, this function takes two arguments - the name and schema of the table.

This schema must meet the following criteria:
- It must contain a list of columns.
- All columns must have either a `pytype` or a `qtype` specified, except the column of vectors.
- One column of vector embeddings may also have a `vectorIndex` attribute with the configuration of the index for similarity search - this column is implicitly an array of `float32`.

### Define Schema

Our table will have two columns the first `id` with a list of dummy ID's, the second will be the vector embeddings we will use for similarity search later on in this example.

We will define our dimensionality, similarity metric and index type with the `vectorIndex` attribute. For this example we chose:
- `dims = 384` : In the next section, we generate embeddings that are 384-dimensional to match this. The number of dimensions should mirror the output dimensions of your embedding model.
- `metric = L2` : We chose [L2/Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance). Our dummy dataset is low dimensional which Euclidean distance is suitable for. You have the choice of using other metrics here like [IP/Inner Product](https://en.wikipedia.org/wiki/Inner_product_space) and [CS/Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) and the one you chose depends on the specific context and nature of your data.
- `type = hnsw` : [HNSW](https://faiss.ai/cpp_api/struct/structfaiss_1_1IndexHNSW.html) enhances efficiency while maintaining accuracy. You have the choice of using other indexes like  and [IVFPQ](https://faiss.ai/cpp_api/struct/structfaiss_1_1IndexIVFPQ.html) and a [Flat index](https://faiss.ai/cpp_api/struct/structfaiss_1_1IndexFlat.html) here, as with metrics the one you chose depends your data and your overall performance requirements.


In [16]:
schema = {
    "columns": [
        {"name": "id", "pytype": "str"},
        {"name": "name", "pytype": "str"},
        {"name": "description", "pytype": "str"},
        {"name": "summary", "pytype": "str"},
        {"name": "title", "pytype": "str"},
        {"name": "visitors", "pytype": "int32"},
        {"name": "description_embedding", "vectorIndex": {"dims": 384, "metric": "L2", "type": "hnsw"}},
    ]
}

### Create Table

In [17]:
table = session.create_table("ai_tools", schema)

## 3. Add Data to the KDB.AI Table

First, generate a vector of five 8-dimensional vectors which will be the vector embeddings in this example. We will then add these to pandas dataframe with column names/types matching the target table.

In [18]:
import requests

gist_url = "https://gist.github.com/mrmps/2f62a2287cb2c1ca63a2762fcaac89bc/raw"
response = requests.get(gist_url)
ai_tools_data = response.json()
df = pd.DataFrame.from_dict(ai_tools_data)
df.drop(columns=["xata"], inplace=True)
df.head()

,description,id,name,summary,title,visitors
0,Generate 3D textures for your game in seconds ...,rec_cfn1112cibvc11jnn2qg,TextureLab,TextureLab is a website that provides 3D textu...,Instant And Unique 3D Textures For Your Next G...,23913
1,Luma Labs enables users to explore 3D modeling...,rec_cfn1112cibvc11jnn2r0,lumalabs,Luma Labs is a website that offers an early ex...,Imagine 3D V1.2 (Alpha),456963
2,Make motion capture from video easier and more...,rec_cfn1112cibvc11jnn2rg,plask,Plask is an AI-powered mocap animation tool th...,Ai-Powered Mocap Animation Tool.,90960
3,Get hundreds of interior design ideas for your...,rec_cfn1112cibvc11jnn2s0,AI Room Planner,AI Room Planner is an online platform that uti...,Interior Design By Ai,211540
4,A platform powered by AI to help you create be...,rec_cfn1112cibvc11jnn2sg,AI TWO,AI TWO is a website that provides a platform f...,Aitwo.Co - The Ai-Powered All-In-One Design Pl...,7201


## Use the FastEmbed library to embed every description in the dataset

The FastEmbed library uses the BAAI/bge-small-en-v1.5 library by default. This is what we will use during inference time as well.

In [19]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding()

descriptions = [tool["description"] for tool in ai_tools_data]
embeddings = list(embedding_model.embed(descriptions))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

ort_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

## Insert the data into our KDB.AI table

In [20]:
# Create a DataFrame with the AI tools data
data = pd.DataFrame(ai_tools_data)[["id", "name", "description", "summary", "title", "visitors"]]
data["description_embedding"] = embeddings

# Bulk insert the data into KDB.AI
table.insert(data)

True

## Use the Hugging Face Inference API to embed the query so that it can be used to search our index

In [21]:
# Perform a search using Hugging Face embeddings
import requests

# make sure your URL looks like this to ensure you get instant results, and not a model loading error
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/BAAI/bge-small-en-v1.5"

def generate_query_embedding(text: str) -> list[float]:
    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {HF_TOKEN}"},
        json={"inputs": text}
    )

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")
    return response.json()

query = "AI tool for creating 3D textures"
query_embedding = generate_query_embedding(query)

results = table.search(
    vectors=[query_embedding],
    n=3,
)

results[0]

,id,name,description,summary,title,visitors,description_embedding,__nn_distance
0,rec_cfn1112cibvc11jnn2qg,TextureLab,Generate 3D textures for your game in seconds ...,TextureLab is a website that provides 3D textu...,Instant And Unique 3D Textures For Your Next G...,23913,"[-0.06803064, 0.017616907, 0.07126939, 0.00746...",0.252240
1,rec_cfn11a2cibvc11jnndbg,Ponzu.gg,Create realistic 3D images with AI-generated t...,Ponzu is a website that helps 3D artists and d...,Ponzu.,6526,"[-0.06462157, -0.014813937, 0.03470334, 0.0305...",0.267235
2,rec_cfn119acibvc11jnncf0,Masterpiece Studio,Create 3D models with Generative AI and deploy...,Masterpiece Studio is a company that has devel...,Masterpiece Studio.,38954,"[-0.04126825, -0.0036431153, 0.059568573, -0.0...",0.342896


## 6. Delete the KDB.AI Table to Conserve Resources


We can use `table.drop()` to delete a table.

In [22]:
table.drop()

True

<div class="alert alert-block alert-warning">
<b>Warning:</b> Once you drop a table, you cannot use it again.
</div>

## Next Steps

Now that you’re successfully making indexes with KDB.AI, you can start inserting your own data or view more examples:
- [PDF Document Search](../document_search)
- [MRI Image Search](../image_search)
- [Music Recommendation System](../music_recommendation)
- [Sensor Pattern Matching](../pattern_matching)
- [Retrieval Augmented Generation with LangChain](../retrieval_augmented_generation)
- [Sentiment Analysis of Reviews](../sentiment_analysis)